## Introduction

For this project, we will attempt what is know as a model extraction, or model stealing attack. In the real world, this is typically performed to help an adversary understand and/or exploit a black-box model.

Here, we will target a model designed expressly for this purpose. "HackThisAI" (https://github.com/JosephTLucas/HackThisAI/) is a series of machine learning hacking challenges, many of which debuted at the infamous Defcon hacker conference in 2021.

We have selected the "Stonks" challenge (https://github.com/JosephTLucas/HackThisAI/tree/main/challenge/medium_stonks), which is a classification model that classifies stocks with a "buy" or "sell" recommendation. This is rated a "medium" difficulty challenge. The model has three features:

OC - Open price minus close price (per day)
HL - High price minus low price (per day)
VOL - Daily volume
To simulate a black-box model to attack, the challenge includes a downloadable Docker image. We have mounted this image and can interact with it via API calls.

This "HackThisAI" challenge is set up as a "Capture the Flag" or "CTF" event. If we successfully duplicate the model, there is an API on the Docker image for submitting our model to receive the "flag" (text string) and get credit for completing the challenge.

## Load and Import Libraries

In [ ]:
!pip install autogluon.tabular[all]
from autogluon.tabular import TabularPredictor
import requests
import pandas as pd
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

## Target ML Model Setup

First we'll set up Docker in Colab (credit to https://github.com/drengskapur/docker-in-colab/blob/main/README.md).

In [2]:
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


Now we'll go retrieve our target model from GitHub.

Because full Docker isn't supported in Google Colab, we need to build the Docket image on a local laptop. I used these commands (on local laptop, not in Colab):

```
wget https://github.com/hotpacket/HackThisAI/archive/refs/heads/main.zip
unzip main.zip
cd HackThisAI/challenge/medium_stonks/
docker build --tag stonks .
docker save -o stonks.tar stonks
```
The orginal challenge can be found here (https://github.com/JosephTLucas/HackThisAI). Credit for creating the challenge goes to Joe Lucas. I modified the original challenge to accept AutoGluOn models for scoring and also fixed a couple tiny bugs in my fork.

Next you can upload stonks.tar to a directory in the Collab filesystem by clicking the folder icon to the left of this text, right clicking on a folder (I chose /tmp/), and choosing Upload. It takes a few minutes to upload.



In [7]:
ls -al /tmp/

total 1041076
drwxrwxrwt 1 root root       4096 Feb 15 14:59 ./
drwxr-xr-x 1 root root       4096 Feb 15 14:11 ../
-rw-r--r-- 1 root root       1183 Feb 15 14:11 dap_multiplexer.a134f0ab95d4.root.log.INFO.20240215-141126.81
lrwxrwxrwx 1 root root         61 Feb 15 14:11 dap_multiplexer.INFO -> dap_multiplexer.a134f0ab95d4.root.log.INFO.20240215-141126.81
srwxr-xr-x 1 root root          0 Feb 15 14:11 debugger_8o76e8k34=
drwx------ 2 root root       4096 Feb 15 14:11 initgoogle_syslog_dir.0/
drwxr-xr-x 2 root root       4096 Feb 15 14:58 .ipynb_checkpoints/
-rw-r--r-- 1 root root      21625 Feb 15 14:58 language_service.a134f0ab95d4.root.log.ERROR.20240215-145831.11365
-rw-r--r-- 1 root root       1990 Feb 15 14:13 language_service.a134f0ab95d4.root.log.INFO.20240215-141142.206
-rw-r--r-- 1 root root       1893 Feb 15 14:58 language_service.a134f0ab95d4.root.log.INFO.20240215-145758.11365
-rw-r--r-- 1 root root        358 Feb 15 14:58 language_service.a134f0ab95d4.root.log.INFO.20240215

Wait until the above command shows the stonks.tar fully uploaded.

Now let's load the stonks.tar as a Docker image.

In [8]:
!runuser -l user -c 'udocker load -i /tmp/stonks.tar'

Info: creating repo: /home/user/.udocker
Info: udocker command line interface 1.3.13
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful
Info: adding layer: sha256:190f3345a0e48807cbb1af6725c8c9d81befe6501cea63bcaeaec30932223e5e
Info: adding layer: sha256:43adf751e55483ea1e7d691d932c1693f3ce4d6beeb4be1ca2cc087863cd67e7
Info: adding layer: sha256:18011d75ffa0586dec6d08049478dd7c75d748d1c560e189b04f2b3bcb09a0a3
Info: adding layer: sha256:9edd0a3360eb70d9c0948543e015067fd772e13791386ae27bffa18af9288b24
Info: adding layer: sha256:55d8ca983303cf9cdbf5847e1bc98952b29457da826c34d204f0a9002c22dbf7
Info: adding layer: sha256:c048ff8289542f7815df897626619b75a157c087d4b01b6c622dea58819429aa
Info: adding layer: sha256:8e787450c5d49f8f9b8e39dcfdf0c34a9c426bde1d43cc46fdeaf4a56c071103
Info: adding layer: sha256:f0f6a7cf9a02ba514171dd904e2d189940df3f016ec4938b749665358402c24b
Info: adding layer: sha256:6c3e7df31590f02f10cb71fc4e

In [9]:
!runuser -l user -c 'udocker images'

REPOSITORY
ce106c2741aa5e04:latest    .


Below, update the name for the Docker image to create the Docker container.

In [10]:
!runuser -l user -c 'udocker create --name=stonks2-target ce106c2741aa5e04:latest'

21daac34-d6cb-3f44-be30-00302b83c2f0


Start terminal (click button on the lower left, need $10 Colab Pro subscription), type in:


```
# su user
$ udocker run c5befa31-a4b9-32b0-ba42-9285d34e95ff <use your container name>
root@476e6d76bb84:/app# pip install autogluon.tabular[all]
root@476e6d76bb84:/app# python3 challenge.py
```

Now the model is running on localhost 5000 and we can interact with it.


In [12]:
r = requests.post("http://localhost:5000/predict", json={"oc": 1, "hl": 2, "vol": 3})
print(r.text)

-1


The "-1" is a response from the target model. It's working!

## Build Dataset to Feed Target Model

Now we need some test data to feed the target model to gather the target model's labels (1 and -1, which correspond to "buy" and "sell" recommendations).

Using Yahoo Finance, I gathered four years of Tesla stock data and saved it as a CSV. I then uploaded TSLA.csv to Google Colab.

We'll import the data into a Pandas dataframe and then format it the way the target model wants it.

In [13]:
import pandas as pd
df = pd.DataFrame(pd.read_csv('/tmp/TSLA.csv'))

newdf = pd.DataFrame()

newdf['oc'] = df['Open'] - df['Close']
newdf['hl'] = df['High'] - df['Low']
newdf['vol'] = df[['Volume']]

newdf

,oc,hl,vol
0,-0.026001,0.384664,78012000
1,-0.225334,0.273333,58573500
2,0.061333,0.404665,62526000
3,0.123333,0.486668,107131500
4,0.705334,0.849333,133638000
...,...,...,...
1253,0.599991,7.110000,111535200
1254,-0.559998,6.039993,83034000
1255,-3.390014,4.639999,84327600
1256,3.979996,7.449997,95498600


Now our data is ready to be sent to the target model.

##Send Data to Target Model and Record Labels

Let's send the data and get the labels!

Due to a limitation placed in the black-box model (likely for availability reasons during the Defcon conference), queries are only accepted once per second. Therefore, we sleep for one second between queries and the following cell takes about 21 minutes to run.

For anyone following along, if you wish to skip this wait, you may skip to four cells below this text, look for "load results from local file."

In [14]:
import time

y = []
for i in range(0, len(newdf)):
    r = requests.post("http://localhost:5000/predict", json={"oc": float(newdf.iloc[i,0]), "hl": float(newdf.iloc[i,1]), "vol": float(newdf.iloc[i,2])})
    y.append(int(r.text))
    time.sleep(1)


print(y)

[1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Next, we add the buy/sell prediction "y" to our DataFrame.

In [15]:
newdf['y'] = y
newdf

,oc,hl,vol,y
0,-0.026001,0.384664,78012000,1
1,-0.225334,0.273333,58573500,1
2,0.061333,0.404665,62526000,1
3,0.123333,0.486668,107131500,-1
4,0.705334,0.849333,133638000,1
...,...,...,...,...
1253,0.599991,7.110000,111535200,-1
1254,-0.559998,6.039993,83034000,-1
1255,-3.390014,4.639999,84327600,-1
1256,3.979996,7.449997,95498600,-1


Let's save this dataframe so we don't have to wait 17 minutes every time to load the data.

In [16]:
newdf.to_csv('/tmp/stonks-saved.csv', index=False)

Load results from local file (optional)

In [ ]:
# newdf = pd.DataFrame(pd.read_csv('/tmp/stonks-saved.csv'))
# newdf.head()

## Build the Stolen Model

At this point, we have everything we need to build our stolen model. We have our dataframe (newdf) with the three features and the labels from the original model. Let's see how AutoGluOn does.

First, we create a 80/20 train/test split.

In [17]:
train_size = int(0.80 * int(len(newdf)))

train_set = newdf.sample(train_size,random_state=44)
test_set = newdf.drop(train_set.index)
train_set = train_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

train_set.info()
test_set.info()
test_set.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   oc      1006 non-null   float64
 1   hl      1006 non-null   float64
 2   vol     1006 non-null   int64  
 3   y       1006 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 31.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   oc      252 non-null    float64
 1   hl      252 non-null    float64
 2   vol     252 non-null    int64  
 3   y       252 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 8.0 KB


,oc,hl,vol,y
0,0.810001,1.015333,343671000,1
1,-0.337334,0.619334,102670500,-1
2,0.434000,0.716667,154215000,1
3,0.001999,0.656000,177009000,1
4,0.536669,0.586666,131184000,1


Next, we train our model with AutoGluOn.

In [18]:
predictor = TabularPredictor(label='y').fit(train_set, presets='best_quality', time_limit=600)

No path specified. Models will be saved in: "AutogluonModels/ag-20240215_161738"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240215_161738/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 150s
AutoGluon will save models to "AutogluonModels/ag-20240215_161738/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0

Several of the candidate models, including the one selected ("weightedEnsemble_L2") have 1.0 (100%) accuracy on the training data. Let's try the test dataset also.

In [19]:
predictor.leaderboard(test_set)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_r177_BAG_L1,1.000000,1.000000,accuracy,0.008281,0.037993,23.570364,0.008281,0.037993,23.570364,1,True,14
1,LightGBMLarge_BAG_L1,1.000000,0.998012,accuracy,0.008561,0.014489,27.470043,0.008561,0.014489,27.470043,1,True,13
2,LightGBM_BAG_L1,1.000000,0.998012,accuracy,0.011276,0.015936,25.529410,0.011276,0.015936,25.529410,1,True,4
3,CatBoost_BAG_L1,1.000000,1.000000,accuracy,0.016988,0.008752,22.699288,0.016988,0.008752,22.699288,1,True,7
4,KNeighborsDist_BAG_L1,1.000000,0.998012,accuracy,0.017108,0.014774,0.004327,0.017108,0.014774,0.004327,1,True,2
5,KNeighborsUnif_BAG_L1,1.000000,0.998012,accuracy,0.017550,0.018641,0.006372,0.017550,0.018641,0.006372,1,True,1
6,WeightedEnsemble_L2,1.000000,1.000000,accuracy,0.018815,0.011743,24.779296,0.001827,0.002991,2.080008,2,True,15
7,WeightedEnsemble_L3,1.000000,1.000000,accuracy,0.018848,0.013446,25.886239,0.001861,0.004694,3.186951,3,True,24
8,XGBoost_BAG_L1,1.000000,0.995030,accuracy,0.083141,0.064077,17.247140,0.083141,0.064077,17.247140,1,True,11
9,RandomForestEntr_BAG_L1,1.000000,0.996024,accuracy,0.086330,0.111346,1.032979,0.086330,0.111346,1.032979,1,True,6


100% accuracy on the test data! We have created an exact copy of the target model!

## Submit the Stolen Model for the Flag

Our next step is to submit our stolen model for evaluation. Note that the original "HackThisAI" cannot score AutoGluOn models. For the below cells to work correctly, the modified version (https://github.com/hotpacket/HackThisAI) must be used.

**Hit Ctrl+C and Enter in the terminal window** to stop the challenge.py script so we can update it, then enter the follow commands in the terminal window:

```
mv challenge.py challenge.py.old
apt install wget
wget https://raw.githubusercontent.com/hotpacket/HackThisAI/main/challenge/medium_stonks/challenge.py
python3 challenge.py
```


Now run the cells below to zip up the model and send it to the API. Note you must update the folder name AutogluonModels/ag-20240214_151557 to the one produced by AutoGluOn. See the last line in the cell where we trained the model, similar to this:

```
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240214_151557")
```

In [21]:
import shutil
shutil.make_archive('model', 'zip', 'AutogluonModels/ag-20240215_161738')


'/content/model.zip'

In [22]:
with open("model.zip", "rb") as f:
    r = requests.post("http://localhost:5000/check", files={"data_file": f})
    print(r.text)

Diamond Hands! FLAG{HODLHODL}
